```
- Copyright 2023 DeepMind Technologies Limited
- All software is licensed under the Apache License, Version 2.0 (Apache 2.0); you may not use this file except in compliance with the Apache 2.0 license. You may obtain a copy of the Apache 2.0 license at: https://www.apache.org/licenses/LICENSE-2.0
- All other materials are licensed under the Creative Commons Attribution 4.0 International License (CC-BY).  You may obtain a copy of the CC-BY license at: https://creativecommons.org/licenses/by/4.0/legalcode
- Unless required by applicable law or agreed to in writing, all software and materials distributed here under the Apache 2.0 or CC-BY licenses are distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the licenses for the specific language governing permissions and limitations under those licenses.
- This is not an official Google product
```

# Cyclic graphs

This notebook contains:
1. the *skeleton* we used for FunSearch to discover large independent sets in the $n$-th strong product of cyclic graphs,
2. the *functions* discovered by FunSearch that construct those independent sets.

## Skeleton

The commented-out decorators are just a way to indicate the main entry point of the program (`@funsearch.run`) and the function that *FunSearch* should evolve (`@funsearch.evolve`).

In [ ]:
"""Obtains maximal independent sets."""
import itertools
import numpy as np


# @funsearch.run
def evaluate(num_nodes: int, n: int) -> int:
  """Returns the size of an independent set."""
  independent_set = solve(num_nodes, n)
  return len(independent_set)


def solve(num_nodes: int, n: int) -> list[tuple[int, ...]]:
  """Gets independent set with maximal size.

  Args:
    num_nodes: The number of nodes of the base cyclic graph.
    n: The power we raise the graph to.

  Returns:
    A list of `n`-tuples in `{0, 1, 2, ..., num_nodes - 1}`.
  """
  to_block = np.array(list(itertools.product([-1, 0, 1], repeat=n)))

  # Powers in decreasing order for compatibility with `itertools.product`, so
  # that the relationship `i = children[i] @ powers` holds for all `i`.
  powers = num_nodes ** np.arange(n - 1, -1, -1)

  # Precompute the priority scores.
  children = np.array(
      list(itertools.product(range(num_nodes), repeat=n)), dtype=np.int32)
  scores = np.array([priority(tuple(child), num_nodes, n)
                     for child in children])

  # Build `max_set` greedily, using scores for prioritization.
  max_set = np.empty(shape=(0, n), dtype=np.int32)
  while np.any(scores != -np.inf):
    # Add a child with a maximum score to `max_set`, and set scores of
    # invalidated children to -inf, so that they never get selected.
    max_index = np.argmax(scores)
    child = children[None, max_index]  # [1, n]

    blocking = np.einsum(
        'cn,n->c', (to_block + child) % num_nodes, powers)  # [C]
    scores[blocking] = -np.inf
    max_set = np.concatenate([max_set, child], axis=0)

  return [tuple(map(int, el)) for el in max_set]


# @funsearch.evolve
def priority(el: tuple[int, ...], num_nodes: int, n: int) -> float:
  """Returns the priority with which we want to add `el` to the set.

  Args:
    el: an n-tuple representing the element to consider whether to add.
    num_nodes: the number of nodes of the base graph.
    n: an integer, power of the graph.

  Returns:
    A number reflecting the priority with which we want to add `el` to the
    independent set.
  """
  return 0.

By executing the skeleton with the trivial `priority` function in place we can check that the resulting independent sets are far from optimal (e.g., the best known construction for the 5th strong product of the 7-node graph has size 367):


In [ ]:
for n in range(1, 6):
  print(n, evaluate(num_nodes=7, n=n))

1 3
2 9
3 27
4 81
5 243


## Discovered function that builds an independent set of size $367$ in $C_7^5$

This matches the size of the best known construction by [Polak & Schrijver (2019)](https://ir.cwi.nl/pub/30364/30364.pdf).

In [ ]:
def priority(el: tuple[int, ...], num_nodes: int, n: int) -> float:
  """Returns the priority with which we want to add `el` to the set."""
  score = 0.
  for i in range(n):
    if el[i] == el[(i + 2) % n]:
      score += 1
    else:
      score -= 1
    x = ((n - 2) * el[i] - el[(i + 1) % n]
         - el[(i + 2) % n] - (n + 1) * el[(i + 3) % n]) % num_nodes
    score -= 0.5 * (x - el[(i + 1) % n]) ** 2
    score += 0.1 * (num_nodes - 1 - (x - 1) % num_nodes) ** 2
    score += 0.2 * (num_nodes - 1 - (x - 2) % num_nodes) ** 2
  return score

In [ ]:
assert evaluate(num_nodes=7, n=5) == 367

## Discovered function that builds the best known independent sets in $C_9^n$ for $n=3,...,7$

These independent sets match the best known construction reported by [Matthew & Östergård (2016)](https://link.springer.com/article/10.1007/s10623-016-0194-7).

In [ ]:
def priority(el: tuple[int, ...], num_nodes: int, n: int) -> float:
  """Returns the priority with which we want to add `el` to the set."""
  s = 0.
  for i in range(n):
    s += el[i] << i
    s %= num_nodes
  return (2 * el[2] - 4 * el[0] + el[1]) % num_nodes + s

Below, we only run the code up until $n=5$. Uncomment the line below to run also the code with $n=6$ and $n=7$, which would take about 10 minutes to execute.

In [ ]:
expected_sizes = {
    3: 81,
    4: 324,
    5: 1458,
    6: 6561,
    7: 26244
}
range_n = range(3, 6)
# range_n = range(3, 8)  # Uncomment to run up until n=7.
for n in range_n:
  assert evaluate(num_nodes=9, n=n) == expected_sizes[n]

## Discovered function that finds an independent set of size 754 in $C_{11}^4$

This is larger than the best known independent set reported by [Matthew & Östergård (2016)](https://link.springer.com/article/10.1007/s10623-016-0194-7), which has size $748$.

In [ ]:
def priority(el: tuple[int, ...], num_nodes: int, n: int) -> float:
  """Returns the priority with which we want to add `el` to the set."""
  el_clipped = np.clip(el, a_min=None, a_max=num_nodes - 3)
  values = 2 * np.array(list(itertools.product(range(1, n), repeat=n)))
  multipliers = np.array(
      [num_nodes ** i for i in range(n - 1, -1, -1)], dtype=np.int32)
  x = np.sum((1 + values + el_clipped) * multipliers, axis=-1)
  return np.sum(x % (num_nodes - 2), dtype=float)

In [ ]:
assert evaluate(num_nodes=11, n=4) == 754